In [2]:
import pandas as pd
import numpy as np
import matplotlib as mp

In [3]:
# Creamos un DataFrame de nuestros datos.
car_data = pd.read_csv('../data/cars.csv')

In [6]:
valores_nulos = car_data.isnull().sum()

valores_nulos

CODE;PRODUCTO;TIPO_CARROCERIA;COMBUSTIBLE;Potencia_;TRANS;FORMA_PAGO;ESTADO_CIVIL;GENERO;OcupaciOn;PROVINCIA;Campanna1;Campanna2;Campanna3;Zona _Renta;REV_Garantia;Averia_grave;QUEJA_CAC;EDAD_COCHE;COSTE_VENTA;km_anno;Mas_1_coche;Revisiones;Edad Cliente;Tiempo    0
dtype: int64

No hay filas comn valores nulos.